In [ ]:
import boto3
import dask
from dask_gateway import Gateway
def get_dask_cluster(number_in_cluster):

    gateway = Gateway()
    running_clusters =  gateway.list_clusters()
    if len(running_clusters) > 0:
        cluster = gateway.connect(running_clusters[0].name)
    else:
        options = gateway.cluster_options()
        session = boto3.Session(profile_name="default")
        credentials = session.get_credentials()
        #options['conda_environment']= 'nebari-git/nebari-git-dask'

        options['conda_environment']= 'nebari-git/nebari-git-dask-analysis'
        options['profile'] = 'Small Worker'
        #options['profile'] = 'Medium Worker'
        options['environment_vars']['AWS_ACCESS_KEY_ID'] = credentials.access_key
        options['environment_vars']['AWS_SECRET_ACCESS_KEY']= credentials.secret_key
        cluster = gateway.new_cluster(options)

        #cluster.adapt(minimum=1,maximum=10)

    dask.config.set({
        "distributed.comm.timeouts.tcp": "1000 seconds",
        "distributed.comm.timeouts.connect": "1000 seconds",
        "distributed.scheduler.worker-ttl": "500 minutes"
    }) 
    cluster.scale(number_in_cluster)
    #cluster.adapt(minimum=1,maximum=20)
    return cluster.get_client()



    